In [8]:
from pathlib import Path
import duckdb
import pandas as pd
# Define the path to the Parquet file
crs_file = Path("data") / "CRS.parquet"
dac1_file = Path("data") / "Table1_Data.csv"
rf_file = Path("data") / "riomarkers.csv"
imputed_multilateral_file = Path("data") / "seek_sectors_revised_with_channel.parquet"
climate_riomarker_file = Path("data") / "climate_riomarkers.txt"
multisystem_file = Path("data") / "multisystem.txt"

pd.set_option('display.max_rows', 100)  # Set a specific number of rows
pd.set_option('display.width', 1000)       # Adjust overall display width
pd.set_option('display.max_colwidth', 500)


data_save_path = Path("../") 

DAC_COUNTRIES = [
  "Australia", "Austria", "Belgium", "Canada", "Czechia", "Denmark", "Finland", "France", "Germany", "Greece", "Hungary",
  "Iceland", "Ireland", "Italy", "Japan", "Korea", "Luxembourg", "Netherlands", "New Zealand", "Norway", "Poland",
  "Portugal", "Slovak Republic", "Slovenia", "Spain", "Sweden", "Switzerland", "United Kingdom", "United States", "Estonia", "Lithuania"
]

In [9]:
#Function to get the available columns in a file
def print_column_names(file):
    with duckdb.connect() as conn:

        cols_query = f"""
        SELECT * FROM '{file.as_posix()}'
        LIMIT 1
        """
        result = conn.execute(cols_query)

        # Get the column names from the description attribute
        column_names = [desc[0] for desc in result.description]

    return column_names

## CRS DATA

In [10]:
print_column_names(crs_file)

['year',
 'donor_code',
 'de_donorcode',
 'donor_name',
 'agency_code',
 'agency_name',
 'crs_id',
 'project_number',
 'initial_report',
 'recipient_code',
 'de_recipientcode',
 'recipient_name',
 'region_code',
 'de_regioncode',
 'region_name',
 'incomegroup_code',
 'de_incomegroupcode',
 'incomegroup_name',
 'flow_code',
 'flow_name',
 'bi_multi',
 'category',
 'finance_t',
 'aid_t',
 'usd_commitment',
 'usd_disbursement',
 'usd_received',
 'usd_commitment_defl',
 'usd_disbursement_defl',
 'usd_received_defl',
 'usd_adjustment',
 'usd_adjustment_defl',
 'usd_amount_untied',
 'usd_amount_partial_tied',
 'usd_amount_tied',
 'usd_amount_untied_defl',
 'usd_amount_partial_tied_defl',
 'usd_amounttied_defl',
 'usd_irtc',
 'usd_expert_commitment',
 'usd_expert_extended',
 'usd_export_credit',
 'currency_code',
 'commitment_national',
 'disbursement_national',
 'grant_equiv',
 'usd_grant_equiv',
 'short_description',
 'project_title',
 'purpose_code',
 'purpose_name',
 'sector_code',
 'sect

# DAC1 DATA

In [11]:
print_column_names(dac1_file)

['DONOR',
 'Donor_1',
 'PART',
 'Part_1',
 'AIDTYPE',
 'Aid type',
 'FLOWS',
 'Fund flows',
 'AMOUNTTYPE',
 'Amount type',
 'TIME',
 'Year',
 'Value',
 'Flags']

## ONE Campaign Multilateral File

In [12]:
print_column_names(imputed_multilateral_file)

['year',
 'indicator',
 'donor_code',
 'donor_name',
 'purpose_code',
 'purpose_name',
 'channel_code',
 'mapped_name',
 'prices',
 'currency',
 'value',
 '__index_level_0__']

## Rio Markers File (Allocable ODA)

In [13]:
print_column_names(rf_file)

['DATAFLOW',
 'STRUCTURE_NAME',
 'ACTION',
 'DONOR',
 'Donor_1',
 'RECIPIENT',
 'Recipient_1',
 'SECTOR',
 'Sector_1',
 'MEASURE',
 'Measure_1',
 'ALLOCABLE',
 'Allocable_1',
 'MARKER',
 'Marker_1',
 'SCORE',
 'Score_1',
 'FLOW_TYPE',
 'Flow type',
 'PRICE_BASE',
 'Price base',
 'MD_DIM',
 'Drilldown dimension',
 'MD_ID',
 'Microdata ID',
 'UNIT_MEASURE',
 'Unit of measure',
 'TIME_PERIOD',
 'Time period',
 'OBS_VALUE',
 'Observation value',
 'BASE_PER',
 'Base period',
 'OBS_STATUS',
 'Observation status',
 'UNIT_MULT',
 'Unit multiplier',
 'DECIMALS',
 'Decimals_1',
 'DONOR_AGENCY',
 'Donor agency',
 'OECD_ID',
 'OECD ID',
 'DONOR_PROJECT_ID',
 'Donor project ID',
 'REGION',
 'Region_1',
 'INCOME_GROUP',
 'Income group',
 'CHANNELDELIVERY_CODE',
 'Channel of delivery code',
 'CHANNELDELIVERY_NAME',
 'Channel of delivery name',
 'SHORT_DESCRIPTION',
 'Short description',
 'PROJECT_TITLE',
 'Project title',
 'LONG_DESCRIPTION',
 'Long description',
 'GENDER_EQUALITY',
 'Gender equality

In [4]:
with duckdb.connect() as conn:

        cols_query = f"""
        SELECT * FROM read_csv_auto('{climate_riomarker_file}', delim='|', header=True)
        LIMIT1
        """
        result = conn.execute(cols_query)

        # Get the column names from the description attribute
        column_names = [desc[0] for desc in result.description]
print(column_names)

['DATAFLOW,STRUCTURE_NAME,ACTION,DONOR,Donor,RECIPIENT,Recipient,SECTOR,Sector,MEASURE,Measure,ALLOCABLE,Allocable,MARKER,Marker,SCORE,Score,FLOW_TYPE,Flow type,PRICE_BASE,Price base,MD_DIM,Drilldown dimension,MD_ID,Microdata ID,UNIT_MEASURE,Unit of measure,TIME_PERIOD,Time period,OBS_VALUE,Observation value,BASE_PER,Base period,OBS_STATUS,Observation status,UNIT_MULT,Unit multiplier,DECIMALS,Decimals,DONOR_AGENCY,Donor agency,OECD_ID,OECD ID,DONOR_PROJECT_ID,Donor project ID,REGION,Region,INCOME_GROUP,Income group,CHANNELDELIVERY_CODE,Channel of delivery code,CHANNELDELIVERY_NAME,Channel of delivery name,SHORT_DESCRIPTION,Short description,PROJECT_TITLE,Project title,LONG_DESCRIPTION,Long description,GENDER_EQUALITY,Gender equality,ENVIRONMENT,Environment,BIODIVERSITY,Biodiversity,CLIMATE_MITIGATION,Climate mitigation,CLIMATE_ADAPTATION,Climate adaptation,DESERTIFICATION,Desertification']


In [5]:
with duckdb.connect() as conn:

        cols_query = f"""
        SELECT * FROM read_csv_auto('{multisystem_file}', delim='|', header=True)
        LIMIT1
        """
        result = conn.execute(cols_query)

        # Get the column names from the description attribute
        column_names = [desc[0] for desc in result.description]
print(column_names)

['Year', 'DonorCode', 'DonorNameE', 'AgencyNameE', 'Amount', 'FlowType', 'AmountType', 'AidToOrThru', 'ChannelCode', 'ChannelNameE', 'ChannelReportedNameE', 'ChannelCategory', 'ChannelCategoryName', 'RecipientCode', 'RecipientNameE', 'Purposecode', 'Purposename_e', 'ShortDescription', 'Gender', 'Environment', 'Free-standing Technical Co-operation', 'Programme Based Approaches', 'InvestmentProject', 'Biodiversity', 'ClimateMitigation', 'ClimateAdaptation', 'Desertification', 'RegionNameE', 'IncomeGroupNameE', 'FlowCode', 'FlowName_e', 'Aid_T', 'Aid_T_Description_E', 'Sector', 'SectorNameE', 'Geography', 'ProjectTitle', 'LongDescription', 'CRSid', 'ProjectNumber']
